### 11단계 가변 길이 인수(순전파 편)

- 가변 길이 입출력을 처리할 수 있도록 확장

#### 11.1 Function 클래스 수정

- 가변 길이 입출력을 표현하려면 변수들을 리스트(또는 튜플)에 넣어 처리하면 편함
- Function 클래스는 지금까지처럼 '하나의 인수'만 받고 '하나의 값'만 반환
- 대신 인수와 반환값의 타입을 리스트로 바꾸고, 필요한 변수들을 이 리스트에 넣음

- 기존의 Function 메서드

In [3]:
class Function:
    def __call__(self, input):
        x = input.data # Variable이라는 상자에서 실제 데이터를 꺼냄
        y = self.forward(x) # forward 메서드에서 구체적인 계산
        output = Variable(as_array(y)) # 계산 결과를 Variable에 넣음
        output.set_creator(self) # 자신이 '창조자'라고 원산지 표시
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

- `__call__`메서드의 인수와 반환값을 리스트로 바꿈

In [7]:
class Function:
    def __call__(self, inputs):
        xs = [x.data for x in inputs]  # Get data from Variable
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]  # Wrap data

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

#### 11.2 Add 클래스 구현

- 인수와 반환값이 리스트(또는 튜플)여야 한다는것을 주의하고 Add 클래스의 forward 메서드 구현

In [24]:
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs # 리스트 xs에서 원소 두개를 꺼냄
        y = x0 + x1
        return (y,) # 튜플형태로 반환(return y, 처럼 괄호는)

- Add 클래스 사용

In [15]:
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
print(y.data)

5


### 12단계 가변 길이 인수(개선 편)

- 첫 번째: Add 클래스를 '사용하는 사람'을 위한 개선
- 두 번째: Add 클래스를 '구현하는 사람'을 위한 개선

#### 12.1 첫 번째 개선: 함수를 사용하기 쉽게

In [19]:
class Function:
    def __call__(self, *inputs): # 별표를 붙이면 리스트를 사용하는 대신 임의 개수의 인수를 건네 함수 호출
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0] # 리스트의 원소가 하나라면 첫 번째 원소를 반환

- 사용 예

In [21]:
def f(*x):
    print(x)

f(1, 2, 3)

(1, 2, 3)


- 이 코드로 알 수 있듯이 함수를 '정의'할 때 인수에 별표를 붙이면 호출할 때 넘긴 인수들을 별표를 붙인 인수 하나로 모아서 받을 수 있다.

In [25]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
print(y.data)

5


#### 12.2 두 번째 개선: 함수를 구현하기 쉽도록

In [29]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) # 별표를 붙여 언택
        if not isinstance(ys, tuple): # 튜플이 아닌 경우 추가
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

In [30]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

#### 12.3 add 함수 구현

- Add 클래스를 '파이썬 함수'로 사용할 수 있는 코드를 추가

In [32]:
def add(x0, x1):
    return Add()(x0, x1)

In [33]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

5


### 13단계 가변 길이 인수(역전파 편)

#### 13.1 가변 길이 인수에 대응한 Add 클래스의 역전파

- 입력 변수가 여러개인 함수를 다변수 함수라 함
- 다변수 함수에서 하나의 입력 변수에만 주목하여(다른 변수는 상수로 취급) 미분하는 것을 편미분이라 함

In [35]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy): # 입력 1개
        return gy, gy # 출력 2개

- 여러 개의 값을 반환할 수 있게 하려면 Variable 클래스의 backward 메서드를 수정

#### 13.2 Variable 클래스 수정

In [38]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs] # 출력 변수인 outputs에 담겨 있는 미분값들을 리스트에 담음
            gxs = f.backward(*gys) # 함수 f의 역전파를 호출, f.backward(*gys)처럼 인수에 별표를 붙여 호출하여 리스트를 풀어줌
            if not isinstance(gxs, tuple): # gxs가 튜플이 아니면 튜플로 변환
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs): # 역전파로 전파되는 미분값을 Variable의 인스턴스 변수 grad에 저장, gxs와 f.inputs의 각 원소는 서로 대응 관계
                x.grad = gx

                if x.creator is not None:
                    funcs.append(x.creator)

#### 13.3 Square 클래스 구현

In [41]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data # 수정 전 : x = self.input.data
        gx = 2 * x * gy
        return gx

- add 함수와 square 함수를 사용해보자

In [44]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0


### 14단계 같은 변수 반복 사용

- 같은 변수를 반복해서 사용할 셩우 의도대로 동작하지 않을 수 있음, 예) y = add(x, x)

In [46]:
x = Variable(np.array(3.0))
y = add(x, x)
print('y', y.data)

y.backward()
print('x.grad', x.grad)

y 6.0
x.grad 1.0


- 동일한 변수를 사용하여 덧셈을 하면 제대로 미분하지 못함

#### 14.1 문제의 원인

In [48]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                x.grad = gx # 1

                if x.creator is not None:
                    funcs.append(x.creator)


- 1 : 출력 쪽에서 전해지는 미분값을 그대로 대입, 같은 변수를 반복해서 사용하면 전파되는 미분값이 덮어 써짐

#### 14.2 해결책

In [60]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None: # 1
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    funcs.append(x.creator)

- 1 : 미분값이 더해지도록 수정

In [61]:
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print('x.grad', x.grad)

x.grad 2.0


In [62]:
x = Variable(np.array(3.0))
y = add(add(x, x), x)
y.backward()
print('x.grad', x.grad)

x.grad 3.0


#### 14.3 미분값 재설정

In [63]:
# 첫 번째 계산
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print('x.grad', x.grad)

# 두 번째 계산(같은 x를 사용하여 다른 계산을)
y = add(add(x, x), x)
y.backward()
print('x.grad', x.grad)

x.grad 2.0
x.grad 5.0


- 값은 변수를 사용하여 '다른' 계산을 할 경우 계산이 꼬이는 문제 발생

In [66]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def cleargrad(self): # 1
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    funcs.append(x.creator)

- 1 : 미분값을 초기화하는 cleargrad 메서드를 추가

In [70]:
# 첫 번째 계산
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print('x.grad', x.grad)

# 두 번째 계산(같은 x를 사용하여 다른 계산을)
x.cleargrad() # 미분값
y = add(add(x, x), x)
y.backward()
print('x.grad', x.grad)

x.grad 2.0
x.grad 3.0


- 두 번째 backward() 호출 전 cleargrad()호출 하면 변수에 누적된 미분값이 초기화되어 문제 해결

### 15단계 복잡한 계산 그래프(이론 편) 

- 변수와 함수가 꼭 한줄로 연결되리라는 법은 없음
- 같은 변수를 반복해서 사용하거나 여러 변수를 입력받는 함수를 사용하는 계산을 살 후 있음
- 그래프의 '연결된 형태'를 위상(topology)이라 함

#### 15.1 역전파의 올바른 순서

- 